<a href="https://colab.research.google.com/github/dasmiq/cs6200-embeddings/blob/main/embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Cross-Language Word Embeddings

In class, we discussed how we can reduce the dimensionality of word representations from their original vector space to an embedding space on the order of a few hundred dimensions. Different modeling choices for word embeddings may be ultimately evaluated by the effectiveness of classification or retrieval models.

In this assignment, however, we will consider another common method of evaluating word embeddings: by judging the usefulness of pairwise distances between words in the embedding space.

Follow along with the examples in this notebook, and implement the sections of code flagged with **TODO**.

In [1]:
## If you already have gensim, you can comment this out.
!pip install gensim

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 9.6 MB/s eta 0:00:00
  Attempting uninstall: scipy
    Found existing installation: scipy 1.14.1
    Uninstalling scipy-1.14.1:
      Successfully uninstalled scipy-1.14.1


In [2]:
import gensim
import numpy as np
from gensim.test.utils import datapath, get_tmpfile
from gensim.models import Word2Vec
from gensim.models.word2vec import LineSentence

We'll start by downloading a plain-text version of the plays of William Shakespeare.

In [3]:
!wget http://www.ccs.neu.edu/home/dasmith/courses/cs6200/shakespeare_plays.txt
lines = [s.split() for s in open('shakespeare_plays.txt')]

--2025-03-14 14:27:19--  http://www.ccs.neu.edu/home/dasmith/courses/cs6200/shakespeare_plays.txt
Resolving www.ccs.neu.edu (www.ccs.neu.edu)... 52.70.229.197
Connecting to www.ccs.neu.edu (www.ccs.neu.edu)|52.70.229.197|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4746840 (4.5M) [text/plain]
Saving to: ‘shakespeare_plays.txt’

shakespeare_plays.t 100%[===================>]   4.53M  19.4MB/s    in 0.2s    

2025-03-14 14:27:20 (19.4 MB/s) - ‘shakespeare_plays.txt’ saved [4746840/4746840]



Then, we'll estimate a simple word2vec model on the Shakespeare texts.

In [4]:
model = Word2Vec(lines)

Even with such a small training set size, you can perform some standard analogy tasks we discussed in class.

In [5]:
model.wv.most_similar(positive=['king','woman'], negative=['man'])

[('queen', 0.7689252495765686),
 ('prince', 0.7229413986206055),
 ('duke', 0.701236367225647),
 ('clarence', 0.697894811630249),
 ('margaret', 0.690160870552063),
 ('cardinal', 0.6711230874061584),
 ('york', 0.6710780262947083),
 ('suffolk', 0.6641265153884888),
 ('gloucester', 0.6534211039543152),
 ('percy', 0.6519837379455566)]

In other words, we want a vector close to `king` and `woman` but subtracting the dimensions that are important to `man`, i.e., `queen`. Other words are mostly noble titles important in Shakespeare's "history" plays.

For the rest of this assignment, we will focus on finding words with similar embeddings, both within and across languages. For example, what words are similar to the name of the title character of *Othello*?

In [6]:
model.wv.most_similar(positive=['othello'])
#model.wv.most_similar(positive=['brutus'])

[('iago', 0.9706300497055054),
 ('desdemona', 0.9623706936836243),
 ('cleopatra', 0.9214695692062378),
 ('emilia', 0.919443666934967),
 ('lear', 0.9131994843482971),
 ('cassio', 0.9121388792991638),
 ('ham', 0.9064014554023743),
 ('cressida', 0.8981956243515015),
 ('rosalind', 0.8947876691818237),
 ('jul', 0.8927379250526428)]

If you know the play, you might see some familiar names.

This search uses cosine similarity. In the default API, you should see the same similarity between the words `othello` and `desdemona` as in the search results above.

In [7]:
model.wv.similarity('othello', 'desdemona')

0.96237075

**TODO**: Your **first task**, therefore, is to implement your own cosine similarity function so that you can reuse it outside of the context of the gensim model object.

In [ ]:
## TODO: Implement cosim
def cosim(v1, v2):
  ## return cosine similarity between v1 and v2
  return 0

## This should give a result similar to model.wv.similarity:
cosim(model.wv['othello'], model.wv['desdemona'])

## Evaluation

We could collect a lot of human judgments about how similar pairs of words, or pairs of Shakespearean characters, are. Then we could compare different word-embedding models by their ability to replicate these human judgments.

If we extend our ambition to multiple languages, however, we can use a word translation task to evaluate word embeddings.

We will use a subset of [Facebook AI's FastText cross-language embeddings](https://fasttext.cc/docs/en/aligned-vectors.html) for several languages. Your task will be to compare English both to French, and to *one more language* from the following set: Arabic, German, Portuguese, Russian, Spanish, Vietnamese, and Chinese.

In [8]:
!wget http://www.ccs.neu.edu/home/dasmith/courses/cs6200/30k.en.vec
!wget http://www.ccs.neu.edu/home/dasmith/courses/cs6200/30k.fr.vec

# TODO: uncomment at least one of these to work with another language
# !wget http://www.ccs.neu.edu/home/dasmith/courses/cs6200/30k.ar.vec
# !wget http://www.ccs.neu.edu/home/dasmith/courses/cs6200/30k.de.vec
# !wget http://www.ccs.neu.edu/home/dasmith/courses/cs6200/30k.pt.vec
# !wget http://www.ccs.neu.edu/home/dasmith/courses/cs6200/30k.ru.vec
# !wget http://www.ccs.neu.edu/home/dasmith/courses/cs6200/30k.es.vec
# !wget http://www.ccs.neu.edu/home/dasmith/courses/cs6200/30k.vi.vec
# !wget http://www.ccs.neu.edu/home/dasmith/courses/cs6200/30k.zh.vec

--2025-03-14 14:28:07--  http://www.ccs.neu.edu/home/dasmith/courses/cs6200/30k.en.vec
Resolving www.ccs.neu.edu (www.ccs.neu.edu)... 52.70.229.197
Connecting to www.ccs.neu.edu (www.ccs.neu.edu)|52.70.229.197|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 67681172 (65M)
Saving to: ‘30k.en.vec’

30k.en.vec          100%[===================>]  64.54M  69.2MB/s    in 0.9s    

2025-03-14 14:28:08 (69.2 MB/s) - ‘30k.en.vec’ saved [67681172/67681172]

--2025-03-14 14:28:08--  http://www.ccs.neu.edu/home/dasmith/courses/cs6200/30k.fr.vec
Resolving www.ccs.neu.edu (www.ccs.neu.edu)... 52.70.229.197
Connecting to www.ccs.neu.edu (www.ccs.neu.edu)|52.70.229.197|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 67802327 (65M)
Saving to: ‘30k.fr.vec’

30k.fr.vec          100%[===================>]  64.66M  42.7MB/s    in 1.5s    

2025-03-14 14:28:10 (42.7 MB/s) - ‘30k.fr.vec’ saved [67802327/67802327]



We'll start by loading the word vectors from their textual file format to a dictionary mapping words to numpy arrays.

In [9]:
def vecref(s):
  (word, srec) = s.split(' ', 1)
  return (word, np.fromstring(srec, sep=' '))

def ftvectors(fname):
  return { k:v for (k, v) in [vecref(s) for s in open(fname)] if len(v) > 1}

envec = ftvectors('30k.en.vec')
frvec = ftvectors('30k.fr.vec')

# TODO: load vectors for one more language, such as zhvec (Chinese)
# arvec = ftvectors('30k.ar.vec')
# devec = ftvectors('30k.de.vec')
# ptvec = ftvectors('30k.pt.vec')
# ruvec = ftvectors('30k.ru.vec')
# esvec = ftvectors('30k.es.vec')
# vivec = ftvectors('30k.vi.vec')
# zhvec = ftvectors('30k.zh.vec')

**TODO**: Your next task is to write a simple function that takes a vector and a dictionary of vectors and finds the most similar item in the dictionary. For this part of the assignment, a linear scan through the dictionary using your `cosim` function from above is acceptible.

In [ ]:
## TODO: implement this search function
def mostSimilar(vec, vecDict):
  ## Use your cosim function from above
  mostSimilar = ''
  similarity = 0
  return (mostSimilar, similarity)

## some example searches
[mostSimilar(envec[e], frvec) for e in ['computer', 'germany', 'matrix', 'physics', 'yeast']]

[('', 0), ('', 0), ('', 0), ('', 0), ('', 0)]

Some matches make more sense than others. Note that `computer` most closely matches `informatique`, the French term for *computer science*. If you looked further down the list, you would see `ordinateur`, the term for *computer*. This is one weakness of a focus only on embeddings for word *types* independent of context.

To evalute cross-language embeddings more broadly, we'll look at a dataset of links between Wikipedia articles.

In [11]:
!wget http://www.ccs.neu.edu/home/dasmith/courses/cs6200/links.tab
links = [s.split() for s in open('links.tab')]

--2025-03-14 14:29:35--  http://www.ccs.neu.edu/home/dasmith/courses/cs6200/links.tab
Resolving www.ccs.neu.edu (www.ccs.neu.edu)... 52.70.229.197
Connecting to www.ccs.neu.edu (www.ccs.neu.edu)|52.70.229.197|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1408915 (1.3M)
Saving to: ‘links.tab’

links.tab           100%[===================>]   1.34M  7.45MB/s    in 0.2s    

2025-03-14 14:29:36 (7.45 MB/s) - ‘links.tab’ saved [1408915/1408915]



This `links` variable consists of triples of `(English term, language, term in that language)`. For example, here is the link between English `academy` and French `académie`:

In [12]:
links[302]

['academy', 'fr', 'académie']

We construct a test set for English-French from the first 1000 links between those languages.

In [13]:
frtest = [x for x in links if x[1] == "fr"][0:1000]
frtest[0:10]

[['aalborg', 'fr', 'aalborg'],
 ['aarhus', 'fr', 'aarhus'],
 ['aba', 'fr', 'aba'],
 ['abad', 'fr', 'abad'],
 ['abandon', 'fr', 'abandon'],
 ['abbas', 'fr', 'abbas'],
 ['abbasid', 'fr', 'abbassides'],
 ['abbess', 'fr', 'abbesse'],
 ['abbey', 'fr', 'abbaye'],
 ['abbot', 'fr', 'abbé']]

**TODO**: Evaluate the English and French embeddings by computing the proportion of English Wikipedia articles whose corresponding French article in this test set `frtest` is also the closest word in embedding space. Skip English articles not covered by the word embedding dictionary.

Since many articles, e.g., about named entities, have the same title in English and French, use the identity function as a baseline and compute its accuracy. In other words, how often would you find the right French articles by simply echoing the English title as if it were French? In the ten example links above, `aalborg` and `aarhus` (two cities in Denmark) are the same in English and French. Remember to iterate only over the 1000 linked Wikipedia articles in the test set, not the entire embedding dictionary.

In [ ]:
## TODO: Compute English-French Wikipedia retrieval accuracy.
accuracy = 0
baselineAccuracy = 0

**TODO**: Compute the accuracy, i.e. precision@1, of the embeddings and of the identity function for the first 1000 links between English and another language besides French. Although the baseline will be lower for languages not written in the Roman alphabet (i.e., Arabic or Chinese), there are still many articles in those languages with headwords written in Roman characters.

In [ ]:
## TODO: Compute English-X Wikipedia retrieval accuracy.


**TODO**: Find the 10 nearest neighbors of each English term to compute "recall at 10" and "mean reciprocal rank at 10".

In [ ]:
## TODO: Compute recall@10 and MRR@10 when retrieving 10 nearest neighbors in French and some other language.

## Speeding up Vector Search (required for CS6200, 20 points extra for IS4200)

The list of Wikipedia headwords is short enough that a linear scan through the non-English language embeddings takes some time but is feasible. In a production system, you could index the word embeddings using SimHash or some other locality sensitive hashing scheme, as we discussed for duplicate detection, to speed up this process.

A relatively easy way to get started with fast vector similarity search is to install Meta's `faiss` (Facebook AI Similarity Search) package and read [the tutorial](https://github.com/facebookresearch/faiss/wiki/Getting-started).

In [14]:
# Outside of colab, you may need a different package manager.
# !apt install libomp-dev
!pip install --upgrade faiss-cpu
# Use this line instead if you have a GPU.
# !python -m pip install --upgrade faiss-gpu
import faiss

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 47.2 MB/s eta 0:00:00


**TODO**: Create two vector indexes, for the FastText embeddings of French and for the other language you evaluated above. Use `faiss` to find the 10 nearest neighbors for the top 1000 Wikipedia headwords you evaluated for English-French and the English-X as above.

First, measure the _effectiveness_ of this approximate vector search approach. How does the R@10 and MRR@10 using `faiss` compare to the brute-force search you did above?

Second, measure the _efficiency_ of this approach. How long in seconds does finding nearest neighbors for 1000 headwords by brute force compare to using `faiss`? (For this exercise, don't worry about amortizing indexing costs.)